In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from itertools import product

import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)


In [2]:
print(torch.__version__)
print(torchvision.__version__)

1.11.0
0.12.0


In [5]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels =1,out_channels = 6,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels =6,out_channels = 12,kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2 = nn.Linear(in_features=120,out_features=60)
        self.out = nn.Linear(in_features=60,out_features=10)

    def forward(self,t):
        #t = t #第一层输入
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size =2,stride=2)

        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t,kernel_size = 2,stride =2)

        t = t.reshape(-1,12*4*4)
        t = F.relu(self.fc1(t))

        t = F.relu(self.fc2(t))

        t = self.out(t)

        return t


In [6]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [7]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/'
    ,train=True
    ,download=True
    ,transform = transforms.Compose([
        transforms.ToTensor()
    ])
)

## 嵌套迭代

In [25]:
batch_size_list = [100, 1000, 10000]
lr_list = [.01, .001, .0001, .00001]

for batch_size in batch_size_list:
    for lr in lr_list:
        network = Network()

        train_loader = torch.utils.data.DataLoader(
            train_set, batch_size=batch_size
        )
        optimizer = optim.Adam(
            network.parameters(), lr=lr
        )

        images, labels = next(iter(train_loader))
        grid = torchvision.utils.make_grid(images)

        comment=f' batch_size={batch_size} lr={lr}'
        tb = SummaryWriter(comment=comment)
        tb.add_image('images', grid)
        tb.add_graph(network, images)

        for epoch in range(5):
            total_loss = 0
            total_correct = 0
            for batch in train_loader:
                images, labels = batch # Get Batch
                preds = network(images) # Pass Batch
                loss = F.cross_entropy(preds, labels) # Calculate Loss
                optimizer.zero_grad() # Zero Gradients
                loss.backward() # Calculate Gradients
                optimizer.step() # Update Weights

                total_loss += loss.item() * batch_size
                total_correct += get_num_correct(preds, labels)

            tb.add_scalar(
                'Loss', total_loss, epoch
            )
            tb.add_scalar(
                'Number Correct', total_correct, epoch
            )
            tb.add_scalar(
                'Accuracy', total_correct / len(train_set), epoch
            )

            for name, param in network.named_parameters():
                tb.add_histogram(name, param, epoch)
                tb.add_histogram(f'{name}.grad', param.grad, epoch)

            print(
                "epoch", epoch
                ,"total_correct:", total_correct
                ,"loss:", total_loss
            )  
        tb.close()

epoch 0 total_correct: 46413 loss: 35664.55728709698
epoch 1 total_correct: 51241 loss: 23663.543684780598
epoch 2 total_correct: 52090 loss: 21554.387152194977
epoch 3 total_correct: 52542 loss: 20280.790719389915
epoch 4 total_correct: 52607 loss: 20125.938968360424
epoch 0 total_correct: 42355 loss: 46275.923442840576
epoch 1 total_correct: 48414 loss: 30926.167038083076
epoch 2 total_correct: 50421 loss: 26373.330394923687
epoch 3 total_correct: 51440 loss: 23576.00471228361
epoch 4 total_correct: 52081 loss: 21781.836453080177
epoch 0 total_correct: 31997 loss: 83450.43635964394
epoch 1 total_correct: 42256 loss: 48080.176001787186
epoch 2 total_correct: 43812 loss: 43287.657472491264
epoch 3 total_correct: 44680 loss: 40765.75691998005
epoch 4 total_correct: 45349 loss: 39031.01156055927
epoch 0 total_correct: 6139 loss: 137815.8599615097
epoch 1 total_correct: 23518 loss: 134062.07616329193
epoch 2 total_correct: 28548 loss: 116921.44904136658
epoch 3 total_correct: 31769 loss: 

In [10]:
network = Network()

for name, weight in network.named_parameters():
    print(name,weight.shape)


conv1.weight torch.Size([6, 1, 5, 5])
conv1.bias torch.Size([6])
conv2.weight torch.Size([12, 6, 5, 5])
conv2.bias torch.Size([12])
fc1.weight torch.Size([120, 192])
fc1.bias torch.Size([120])
fc2.weight torch.Size([60, 120])
fc2.bias torch.Size([60])
out.weight torch.Size([10, 60])
out.bias torch.Size([10])


In [20]:

for name, weight in network.named_parameters():
    print(f'{name}.grad',weight.grad.shape)

conv1.weight.grad None
conv1.bias.grad None
conv2.weight.grad None
conv2.bias.grad None
fc1.weight.grad None
fc1.bias.grad None
fc2.weight.grad None
fc2.bias.grad None
out.weight.grad None
out.bias.grad None


### 不同参数遍历写法

In [21]:
from itertools import product

In [22]:
#将参数放入一个字典中
parameters = dict(
    lr = [.01,.001]
    ,batch_size = [10,100,1000]
    ,suffle = [True,False]
)

In [23]:
param_values = [v for v in parameters.values()]
param_values

[[0.01, 0.001], [10, 100, 1000], [True, False]]

In [24]:
for lr,batch_size,suffle in product(*param_values):
    print(lr,batch_size,suffle)

0.01 10 True
0.01 10 False
0.01 100 True
0.01 100 False
0.01 1000 True
0.01 1000 False
0.001 10 True
0.001 10 False
0.001 100 True
0.001 100 False
0.001 1000 True
0.001 1000 False


In [ ]:
for lr,batch_size,shuffle in product(*param_values):
    comment = f'batch_size={batch_size} lr = {lr} shuffle ={shuffle}'

In [ ]:
for lr, batch_size, shuffle in product(*param_values): 
    comment = f' batch_size={batch_size} lr={lr} shuffle={shuffle}'

    train_loader = torch.utils.data.DataLoader(
        train_set
        ,batch_size=batch_size
        ,shuffle=shuffle 
    )

    optimizer = optim.Adam(
        network.parameters(), lr=lr
    )

    network = Network()


    images,labels = next(iter(train_loader))
    grid = torchvision.utils.make_grid(images)

    tb = SummaryWriter(comment=comment)
    tb.add_image('images',grid)
    tb.add_graph(network,images)

    for epoch in range(5):
        total_loss = 0
        total_correct = 0
        for batch in train_loader:
            images,labels = batch
            
            preds = network(images)
            loss = F.cross_entropy(preds,labels)
            #------------要将梯度归零 因为pytorch会累加梯度
            optimizer.zero_grad()
            loss.backward()#计算梯度
            optimizer.step()#更新梯度

            total_loss += loss.item() * batch_size
            total_correct += get_num_correct(preds,labels)
    #scalar 数字
        tb.add_scalar('loss',total_loss,epoch)
        tb.add_scalar('Number Correcnt',total_correct,epoch)
        tb.add_scalar('Accuracy',total_correct/len(train_set),epoch)
        
        for name,param in network.named_parameters():
            tb.add_histogram(name,param,epoch)
            tb.add_histogram(f'{name}.grad,param,gard,epoch param')
        
        print("epoch:",epoch,"total_correct",total_correct," loss:",total_loss)

    tb.close()